In [1]:
from torchinfo import summary

In [2]:
from rl_gomoku.common import AlphaZeroValueNet

In [3]:
model = AlphaZeroValueNet(15, 7, use_attention=True)

In [2]:
import numpy as np
import argparse
import os
import json
import time
import random
import tqdm

from rl_gomoku.envs.array_gomoku import ArrayGomoku
from rl_gomoku.utils import start_play, collect_data, start_self_play, PolicyValueNet, collect_self_play_data, replay_data_from_list
from rl_gomoku.agents.mcts_alpha import MCTSZeroAgent
from rl_gomoku.agents import GreedyAgent
from rl_gomoku.envs import GomokuEnv
from collections import deque

In [44]:
env = ArrayGomoku()

In [3]:
with open("./cache_data//Gomoku-2023-9/10001-10100.matches", "r") as f:
    a = f.readlines()

In [78]:
def parse_data(line):
    result = {}
    tmp = list(filter(lambda x: "0" in x or "1" in x, line['log']))
    tmp = [list(x.values())[0]["response"] for x in tmp]
    to_position = lambda x: x["x"] * 15 + x["y"]
    tmp = [to_position(x) for x in tmp]
    result["data"] = tmp
    
    player_0 = line['players'][0]
    player_1 = line['players'][1]
    
    if player_0["type"] == "bot" and player_1["type"] == "bot":
        result["type"] = "bot"
    else:
        result["type"] = "browser"
    result["bot_0"] = player_0["bot"]
    result["bot_1"] = player_1["bot"]
    return result

In [79]:
data = parse_data(json.loads(a[0]))

In [80]:
data

{'data': [112,
  96,
  82,
  98,
  97,
  67,
  111,
  83,
  113,
  114,
  65,
  129,
  49,
  81,
  95,
  99,
  127,
  142,
  143,
  51,
  159],
 'type': 'bot',
 'bot_0': '5f06d7a47e197a05f076eb50',
 'bot_1': '5bcc67530681335cc1f1d8c3'}

In [82]:
json.loads(a[10])["scores"]

[2, 0]

In [87]:
parse_data(json.loads(a[27]))["data"]

[112,
 127,
 96,
 128,
 126,
 98,
 111,
 141,
 113,
 110,
 97,
 81,
 125,
 83,
 114,
 115,
 153,
 139,
 99,
 129,
 130,
 143,
 157,
 142,
 140,
 154,
 169,
 101,
 87,
 82,
 84,
 69,
 85,
 71,
 86,
 88,
 72,
 102,
 80,
 103,
 100,
 58,
 118,
 145,
 144,
 158,
 173,
 73,
 43,
 68,
 67,
 137,
 48,
 64,
 124,
 38,
 53,
 131,
 117,
 108,
 123,
 122,
 138,
 107,
 92,
 109,
 106,
 168,
 152,
 94,
 79,
 54,
 40,
 55,
 41,
 42,
 166,
 180,
 78,
 56,
 57,
 63,
 76,
 77,
 49,
 136,
 150,
 50,
 34,
 62,
 61,
 91,
 116,
 65,
 66,
 148,
 39,
 22,
 36,
 70,
 6,
 33,
 25,
 11,
 21,
 51,
 46,
 47,
 32,
 31,
 147,
 26,
 10,
 162,
 176,
 28,
 14,
 161,
 160,
 193,
 177,
 163,
 175,
 174,
 178,
 179,
 206,
 192,
 205,
 190,
 189,
 221,
 203,
 194,
 191,
 204,
 164,
 201,
 207,
 223,
 209,
 208,
 4,
 7,
 18,
 60,
 222,
 37,
 23,
 156,
 170,
 171,
 186,
 155,
 188,
 185,
 199,
 198,
 5,
 2,
 20,
 17,
 182,
 167,
 45,
 184,
 183,
 202,
 197,
 181,
 200,
 172,
 135,
 120,
 187,
 132,
 133,
 27,
 16,
 1,
 3,
 

In [85]:
[(i, json.loads(a[i])["scores"]) for i in range(len(a))]

[(0, [2, 0]),
 (1, [0, 2]),
 (2, [2, 0]),
 (3, [2, 0]),
 (4, [2, 0]),
 (5, [2, 0]),
 (6, [2, 0]),
 (7, [2, 0]),
 (8, [2, 0]),
 (9, [2, 0]),
 (10, [2, 0]),
 (11, [2, 0]),
 (12, [2, 0]),
 (13, [0, 2]),
 (14, [0, 2]),
 (15, [0, 2]),
 (16, [2, 0]),
 (17, [2, 0]),
 (18, [2, 0]),
 (19, [2, 0]),
 (20, [2, 0]),
 (21, [2, 0]),
 (22, [2, 0]),
 (23, [2, 0]),
 (24, [2, 0]),
 (25, [0, 2]),
 (26, [0, 2]),
 (27, [1, 1]),
 (28, [1, 1]),
 (29, [0, 2]),
 (30, [2, 0]),
 (31, [2, 0]),
 (32, [0, 2]),
 (33, [2, 0]),
 (34, [0, 2]),
 (35, [2, 0]),
 (36, [1, 1]),
 (37, [0, 2]),
 (38, [2, 0]),
 (39, [2, 0]),
 (40, [0, 2]),
 (41, [2, 0]),
 (42, [2, 0]),
 (43, [0, 2]),
 (44, [2, 0]),
 (45, [0, 2]),
 (46, [2, 0]),
 (47, [2, 0]),
 (48, [0, 2]),
 (49, [1, 1]),
 (50, [2, 0]),
 (51, [1, 1]),
 (52, [0, 2]),
 (53, [2, 0]),
 (54, [0, 2]),
 (55, [2, 0]),
 (56, [1, 1]),
 (57, [2, 0]),
 (58, [2, 0]),
 (59, [2, 0]),
 (60, [2, 0]),
 (61, [0, 2]),
 (62, [2, 0]),
 (63, [2, 0]),
 (64, [0, 2]),
 (65, [2, 0]),
 (66, [0, 2]),
 (67,

In [88]:
replay_data_from_list(env, parse_data(json.loads(a[27]))["data"], render=True)

   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14
0  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
1  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
2  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
3  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
4  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
5  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
6  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
7  _  _  _  _  _  _  _  X  _  _  _  _  _  _  _
8  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
9  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
10 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
11 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
12 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
13 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
14 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
last position: 112, player: 2
   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14
0  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
1  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
2  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
3  _  _  _  _  _  _  _  _  _

(-1, <zip at 0x7f8d2e0db4c0>)